In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
import pandas as pd
fever=pd.read_csv("/content/drive/MyDrive/My Work/IR/FEVER.csv")
characters = ['[',"'",',',']']
fever['claim'] = fever['claim'].tolist()
for i in range(len(fever['claim'])):
  fever['claim'][i] = [ word for word in fever['claim'][i] if word not in characters]
model1 = gensim.models.Word2Vec(list(fever['claim']),min_count=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
fever['claim']

0         [n, i, k, o, l, a, j,  , w, o, r, k, e, d,  , ...
1         [r, o, m, a, n,  , a, t, w, o, o, d,  , c, o, ...
2         [h, i, s, t, o, r, y,  , a, r, t,  , i, n, c, ...
3         [a, d, r, i, e, n, n, e,  , b, a, i, l, o, n, ...
4         [h, o, m, e, l, a, n, d,  , a, m, e, r, i, c, ...
                                ...                        
109805    [l, e, d,  , z, e, p, p, e, l, i, n,  , r, e, ...
109806              [t, a, a, l,  , r, o, m, a, n, t, i, c]
109807    [h, e, r,  , s, t, a, r, s,  , a, m, e, r, i, ...
109808    [t, o, l, k, i, e, n,  , c, r, e, a, t, e, d, ...
109809    [s, u, s, a, n,  , s, a, r, a, n, d, o, n,  , ...
Name: claim, Length: 109810, dtype: object

In [ ]:
import torch
list1 = list('roman')+list('atwood')
list2 = list('roman')+ list('atwood')+list('content')+list('creator')
print(list2)
v1 = model1.wv[list1]
v2 = model1.wv[list2]
cossim=torch.nn.CosineSimilarity(dim=0, eps=1e-08)
v1 = torch.norm(torch.from_numpy(v1),dim=0)
v2 = torch.norm(torch.from_numpy(v2),dim=0)
cossim(v1,v2)

['r', 'o', 'm', 'a', 'n', 'a', 't', 'w', 'o', 'o', 'd', 'c', 'o', 'n', 't', 'e', 'n', 't', 'c', 'r', 'e', 'a', 't', 'o', 'r']


tensor(0.9911)

In [ ]:
import json
tweets = []
for line in open("/content/drive/MyDrive/My Work/IR/train.jsonl", 'r'):
    tweets.append(json.loads(line))
evidence = []
for t in tweets:
  evi = []
  if(t['label']=="NOT ENOUGH INFO"):
    continue
  if len(t['evidence'][0]) >1:
    evi.append(t['evidence'][0])
    evi.append(len(t['evidence'][0]))
  else :
    evi.append(t['evidence'][0][0])
    if t['evidence'][0][0][2] is None:
      evi.append(None)
    else: 
      evi.append(len(t['evidence'][0]))
  evidence.append(evi)
print(evidence[:10])

[[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7], [92206, 104971, 'Fox_Broadcasting_Company', 0]], 2], [[174271, 187498, 'Roman_Atwood', 1], 1], [[255136, 254645, 'History_of_art', 2], 1], [[180804, 193183, 'Adrienne_Bailon', 0], 1], [[[151831, 166598, 'Homeland_-LRB-TV_series-RRB-', 0], [151831, 166598, 'Prisoners_of_War_-LRB-TV_series-RRB-', 0]], 2], [[49158, 58489, 'Boston_Celtics', 3], 1], [[23513, 28977, 'The_Ten_Commandments_-LRB-1956_film-RRB-', 0], 1], [[269479, 265800, 'Tetris', 18], 1], [[56492, 66697, 'Cyndi_Lauper', 2], 1], [[93100, 106004, 'The_Hunger_Games_-LRB-film-RRB-', 0], 1]]


In [ ]:
actual = []
for t in tweets:
  if(t['label']=="NOT ENOUGH INFO"):
    continue
  evi = []
  for i in range(len(t['evidence'])):
    t['evidence'][i][0][3] = i+1
    evi.append(t['evidence'][i][0])
  actual.append([evi])

In [ ]:
claim_list = [''.join(c) for c in fever['claim']]

In [ ]:
def calc_cossim(query):
  max = 0
  query_list = []
  doc_retrieve_list = []
  actual = []
  for word in query.lower():
    query_list = query_list+list(word)
  characters = ['[',"'",',',']','0','1','2','3','4','5','6','7','8','9','-','.']
  cossim=torch.nn.CosineSimilarity(dim=0, eps=1e-08)
  query_list = [ word for word in query_list if word not in characters]
  v2 = model1.wv[query_list]
  v2 = torch.norm(torch.from_numpy(v2),dim=0)
  for i in range(len(fever['claim'])):
    try:
      v1 = model1.wv[fever['claim'][i]]
      v1 = torch.norm(torch.from_numpy(v1),dim=0)
      if max< cossim(v1,v2):
        max = cossim(v1,v2)
        if type(evidence[i][0][0]) == list:
          e = []
          for evi in fever['evidence'][i][0]:
            e.append(evi[2])
        else:
          e = evidence[i][0][2]
        doc_retrieve_list.append(e) 
    except:
      pass
  doc_retrieve_list = doc_retrieve_list[::-1]
  doc_retrieve_list = [i for n, i in enumerate(doc_retrieve_list) if i not in doc_retrieve_list[:n]]
  if len(doc_retrieve_list) > 5:
    doc_retrieve_list = doc_retrieve_list[:5]
  return doc_retrieve_list

evi = []
predicted = []
for i in range(10):
  query = claim_list[i]
  
  r = calc_cossim(query)
  if r is not None:
    pred = []
    for i in range(len(r)):
      pred.append([r[i],i+1])
    predicted.append(pred)
predicted = list(filter(lambda a: a != [], predicted))
print(predicted)


[[['Roman_Atwood', 1]], [['History_of_art', 1]], [['Adrienne_Bailon', 1], ['History_of_art', 2]], [['History_of_art', 1]], [['Boston_Celtics', 1], ['History_of_art', 2]], [['The_Ten_Commandments_-LRB-1956_film-RRB-', 1], ['Boston_Celtics', 2], ['History_of_art', 3]], [['Tetris', 1], ['History_of_art', 2]], [['Cyndi_Lauper', 1]], [['The_Hunger_Games_-LRB-film-RRB-', 1], ['Boston_Celtics', 2], ['History_of_art', 3]]]


In [ ]:
actual = actual[1:]

In [ ]:
query = input('Enter query\n')
print(calc_cossim(query))

Enter query
nikolaj
['Nikolaj_Coster-Waldau', 'Nicole_Kidman', 'Nicki_Minaj', 'International_relations', 'Jennifer_Aniston']


In [ ]:
import six

def check_predicted_evidence_format(instance):
    if 'predicted_evidence' in instance.keys() and len(instance['predicted_evidence']):
        assert all(isinstance(prediction, list)
                   for prediction in instance["predicted_evidence"]), \
            "Predicted evidence must be a list of (page,line) lists"

        assert all(len(prediction) == 2
                   for prediction in instance["predicted_evidence"]), \
            "Predicted evidence must be a list of (page,line) lists"

        assert all(isinstance(prediction[0], six.string_types)
                    for prediction in instance["predicted_evidence"]), \
            "Predicted evidence must be a list of (page<string>,line<int>) lists"

        assert all(isinstance(prediction[1], int)
                   for prediction in instance["predicted_evidence"]), \
            "Predicted evidence must be a list of (page<string>,line<int>) lists"


def is_correct_label(instance):
    return instance["label"].upper() == instance["predicted_label"].upper()


def is_strictly_correct(instance, max_evidence=None):
    #Strict evidence matching is only for NEI class
    check_predicted_evidence_format(instance)

    if instance["label"].upper() != "NOT ENOUGH INFO" and is_correct_label(instance):
        assert 'predicted_evidence' in instance, "Predicted evidence must be provided for strict scoring"

        if max_evidence is None:
            max_evidence = len(instance["predicted_evidence"])


        for evience_group in instance["evidence"]:
            #Filter out the annotation ids. We just want the evidence page and line number
            actual_sentences = [[e[2], e[3]] for e in evience_group]
            #Only return true if an entire group of actual sentences is in the predicted sentences
            if all([actual_sent in instance["predicted_evidence"][:max_evidence] for actual_sent in actual_sentences]):
                return True

    #If the class is NEI, we don't score the evidence retrieval component
    elif instance["label"].upper() == "NOT ENOUGH INFO" and is_correct_label(instance):
        return True

    return False


def evidence_macro_precision(instance, max_evidence=None):
    this_precision = 0.0
    this_precision_hits = 0.0

    if instance["label"].upper() != "NOT ENOUGH INFO":
        all_evi = [[e[2], e[3]] for eg in instance["evidence"] for e in eg if e[3] is not None]

        predicted_evidence = instance["predicted_evidence"] if max_evidence is None else \
                                                                        instance["predicted_evidence"][:max_evidence]

        for prediction in predicted_evidence:
            if prediction in all_evi:
                this_precision += 1.0
            this_precision_hits += 1.0

        return (this_precision / this_precision_hits) if this_precision_hits > 0 else 1.0, 1.0

    return 0.0, 0.0

def evidence_macro_recall(instance, max_evidence=None):
    # We only want to score F1/Precision/Recall of recalled evidence for NEI claims
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # If there's no evidence to predict, return 1
        if len(instance["evidence"]) == 0 or all([len(eg) == 0 for eg in instance]):
           return 1.0, 1.0

        predicted_evidence = instance["predicted_evidence"] if max_evidence is None else \
                                                                        instance["predicted_evidence"][:max_evidence]

        for evidence_group in instance["evidence"]:
            evidence = [[e[2], e[3]] for e in evidence_group]
            if all([item in predicted_evidence for item in evidence]):
                # We only want to score complete groups of evidence. Incomplete groups are worthless.
                return 1.0, 1.0
        return 0.0, 1.0
    return 0.0, 0.0


# Micro is not used. This code is just included to demostrate our model of macro/micro
def evidence_micro_precision(instance):
    this_precision = 0
    this_precision_hits = 0

    # We only want to score Macro F1/Precision/Recall of recalled evidence for NEI claims
    if instance["label"].upper() != "NOT ENOUGH INFO":
        all_evi = [[e[2], e[3]] for eg in instance["evidence"] for e in eg if e[3] is not None]

        for prediction in instance["predicted_evidence"]:
            if prediction in all_evi:
                this_precision += 1.0
            this_precision_hits += 1.0

    return this_precision, this_precision_hits


def fever_score(predictions,actual=None, max_evidence=5):
    correct = 0
    strict = 0

    macro_precision = 0
    macro_precision_hits = 0

    macro_recall = 0
    macro_recall_hits = 0

    for idx,instance in enumerate(predictions):
        assert 'predicted_evidence' in instance.keys(), 'evidence must be provided for the prediction'

        #If it's a blind test set, we need to copy in the values from the actual data
        if 'evidence' not in instance or 'label' not in instance:
            assert actual is not None, 'in blind evaluation mode, actual data must be provided'
            assert len(actual) == len(predictions), 'actual data and predicted data length must match'
            assert 'evidence' in actual[idx].keys(), 'evidence must be provided for the actual evidence'
            instance['evidence'] = actual[idx]['evidence']
            instance['label'] = actual[idx]['label']

        assert 'evidence' in instance.keys(), 'gold evidence must be provided'

        if is_correct_label(instance):
            correct += 1.0

            if is_strictly_correct(instance, max_evidence):
                strict+=1.0

        macro_prec = evidence_macro_precision(instance, max_evidence)
        macro_precision += macro_prec[0]
        macro_precision_hits += macro_prec[1]

        macro_rec = evidence_macro_recall(instance, max_evidence)
        macro_recall += macro_rec[0]
        macro_recall_hits += macro_rec[1]

    total = len(predictions)

    strict_score = strict / total
    acc_score = correct / total

    pr = (macro_precision / macro_precision_hits) if macro_precision_hits > 0 else 1.0
    rec = (macro_recall / macro_recall_hits) if macro_recall_hits > 0 else 0.0
    try :
      f1 = 2.0 * pr * rec / (pr + rec)
    except:
      f1 = 0

    return strict_score, acc_score, pr, rec, f1

In [ ]:
label = "SUPPORTS"
scores = []
for i in range(len(predicted)):
  instance = {
      "label":label, 
      "predicted_label":label, 
      "predicted_evidence":predicted[i], 
      "evidence": actual[i]
      }
  strict_score, label_accuracy, precision, recall, f1 = fever_score([instance])
  scores.append([strict_score, label_accuracy, precision, recall, f1])
print(scores)

[[0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0], [0.0, 1.0, 0.0, 0.0, 0]]
